### Luke Schwenke
### Assignment #6 - Image Classification
### May 13, 2023

In [1]:
from keras.preprocessing import image as ki
import numpy as np
import pandas as pd
from keras import backend as k
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, Flatten
from keras.layers import LSTM, Dense, Dropout, MaxPooling2D
from keras.models import model_from_json

## Data Processing

#### Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" with the following parameters: 

* rescale = 1./255
* shear_range = 0.2
* zoom_range = 0.2
* horizontal_flip = True

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

#### Then build your training set by using the method ".flow_from_directory()"

In [3]:
train_datagen = train_datagen.flow_from_directory(
    './dataset_train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 88 images belonging to 4 classes.


In [4]:
print("Image shape:", train_datagen.image_shape)
print("Number of classes:", train_datagen.num_classes)

Image shape: (64, 64, 3)
Number of classes: 4


## Initial Classifier Build

#### Create an instance of Sequential called "classifier"

In [5]:
k.clear_session() # Clear previous session and reset (release GPU memory resources by resetting backend state)
classifier = Sequential() # Instance for Linear sequence model building

#### Add a Conv2D layer with the following parameters: 
* filters = 32
* kernel_size = (3,3)
* input_shape = image shape found in part 1
* activation = relu

In [6]:
classifier.add(Conv2D(filters=32, 
                      kernel_size=(3, 3),
                      input_shape=(64, 64, 3), 
                      activation="relu"))

#### Add a MaxPooling2D layer where pool_size = (2,2)

In [7]:
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#### Add another Conv2D layer: 
* filters = 64
* kernel_size = (3,3)
* activation = relu

In [8]:
classifier.add(Conv2D(filters=32, 
                      kernel_size=(3, 3), 
                      activation="relu"))

#### Add a MaxPooling2D layer where pool_size = (2,2)

In [9]:
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#### Add a Flatten layer

In [10]:
classifier.add(Flatten())

#### Add a Dense layer
* units = 128
* activation = relu

In [11]:
classifier.add(Dense(units=128, 
                     activation="relu"))

#### Add a final Dense layer (this will output our probabilities):
* units = # of classes
* activation = softmax 

In [12]:
classifier.add(Dense(units=train_datagen.num_classes, 
                     activation="softmax"))

#### Compile with the following: 
* optimize = adam
* loss = categorical cross entropy
* metric = accuracy

In [13]:
classifier.compile(optimizer="adam", 
                   loss="categorical_crossentropy", 
                   metrics=["acc"])

classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [14]:
classifier

## Model Runs

#### Use .fit() with the training set. For the first run, use the following parameters: 
* steps_per_epoch = 3
* epochs = 3

In [15]:
classifier_fit = classifier.fit(train_datagen, 
                                steps_per_epoch=3, 
                                epochs=3)

Epoch 1/3
3/3 [==============================] - 1s 149ms/step - loss: 1.2214 - acc: 0.4545
Epoch 2/3
3/3 [==============================] - 1s 152ms/step - loss: 0.8163 - acc: 0.6932
Epoch 3/3
3/3 [==============================] - 1s 149ms/step - loss: 0.4469 - acc: 0.8750


#### Save Model

In [16]:
classifier.save('deeplearning_classifier_saved')
print("Saved model")

INFO:tensorflow:Assets written to: deeplearning_classifier_saved\assets


INFO:tensorflow:Assets written to: deeplearning_classifier_saved\assets


Saved model


####  Predict using the model built in step 2

In [17]:
import os, glob
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('deeplearning_classifier_saved')
print("Loaded model from disk")

# test data path
img_dir = "./dataset_test" # Enter Directory of test set

# iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# print the files in the dataset_test folder 
for f in files:
    print(f)
    
# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

Loaded model from disk
./dataset_test\1022.png
./dataset_test\1053.png
./dataset_test\4011.png
./dataset_test\4053.png
./dataset_test\6023.png
./dataset_test\6051.png
./dataset_test\C014.png
./dataset_test\C033.png
1/1 [==============================] - 0s 14ms/step


[array([0], dtype=int64),
 array([1], dtype=int64),
 array([1], dtype=int64),
 array([2], dtype=int64),
 array([1], dtype=int64),
 array([1], dtype=int64),
 array([1], dtype=int64),
 array([3], dtype=int64)]

#### Determine Accuracy
* **Note**: To determine accuracy, you will need to check the labels given to each class in the training data and manually label your test data. This will require you to
* Look into the training data(images) in the dataset_train folder, and then determine how a category was coded in keras using the following code:

In [18]:
# check category labels in training_set
train_datagen.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

Look in the test data(images) in the dataset_test folder, and identify what category each images belongs to using images in the training set as references(there are only 8 test observations).
Create a list to store the category/labels for the test data as the actual values.

In [19]:
actual_labels = [0, 0, 2, 2, 1, 1, 3, 3] # Hand labelled from the test images

Compare the predicted values to the actual values for the test set and calculate accuracy score

In [20]:
predicted = [a for r in results for a in r]
predicted # Extracted the prediction for each of the 8 arrays above

[0, 1, 1, 2, 1, 1, 1, 3]

In [21]:
correct_predictions = [i for i, j in zip(actual_labels, predicted) if i == j]
correct_predictions # Keep the pairwise combinations that are equal -- these are the correct predictions

[0, 2, 1, 1, 3]

In [22]:
accuracy = len(correct_predictions) / len(actual_labels) # Number of correct / Total
print(f"Model accuracy: {accuracy * 100}%\n")

Model accuracy: 62.5%



#### Run this same process over a grid of combinations

In [23]:
def fit_and_calculate(params):
    classifier = Sequential()
    classifier.add(Conv2D(32, (3, 3), activation="relu",input_shape=(64,64,3)))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Conv2D(64, (3, 3), activation="relu"))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(128, activation='relu'))
    classifier.add(Dense(4, activation='softmax'))
    classifier.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=["acc"])
    # Fit to the unique parameters in the grid
    classifier.fit(train_datagen, 
                   steps_per_epoch=params["steps_per_epoch"], 
                   epochs=params["epochs"])
    # Save this model off
    classifier.save(params["model_name"])
    print("Saved model")
    
    import os, glob
    import numpy as np
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.models import load_model

    # returns a compiled model
    # identical to the previous one
    model = load_model(params["model_name"])
    print("Loaded model from disk")

    # test data path
    img_dir = "./dataset_test"

    # iterate over each test image
    data_path = os.path.join(img_dir, '*g')
    files = glob.glob(data_path)
    
    data = []
    results = []
    for f1 in files:
        img = image.load_img(f1, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)
        
    test_label= [[0], [0], [2], [2], [1], [1], [3], [3]]
    predicted = [a for b in results for a in b]
    test_label == predicted
    accuracy = len([a for a, b in zip(test_label, predicted) if a == b]) / len(test_label)
    print(f"Model accuracy: {accuracy * 100}%\n")
    return(accuracy)


In [24]:
param_grid = [{"steps_per_epoch": 1, "epochs": 1, "model_name": "model_1"},
              {"steps_per_epoch": 1, "epochs": 2, "model_name": "model_2"},
              {"steps_per_epoch": 1, "epochs": 3, "model_name": "model_3"},
              {"steps_per_epoch": 2, "epochs": 4, "model_name": "model_4"},
              {"steps_per_epoch": 2, "epochs": 5, "model_name": "model_5"},
              {"steps_per_epoch": 2, "epochs": 6, "model_name": "model_6"},
              {"steps_per_epoch": 3, "epochs": 7, "model_name": "model_7"},
              {"steps_per_epoch": 3, "epochs": 8, "model_name": "model_8"},
              {"steps_per_epoch": 5, "epochs": 9, "model_name": "model_9"},
              {"steps_per_epoch": 5, "epochs": 10, "model_name": "model_10"}]

In [25]:
accuracies = []
for params in param_grid:
    acc = fit_and_calculate(params)
    accuracies.append(acc)
    print(f"Accuracy for {params['model_name']}: {acc}")
    
# print out the final accuracies list
print("Accuracies for all models:", accuracies)

1/1 [==============================] - 1s 665ms/step - loss: 1.3414 - acc: 0.3333


INFO:tensorflow:Assets written to: model_1\assets


INFO:tensorflow:Assets written to: model_1\assets


Saved model
Loaded model from disk
1/1 [==============================] - 0s 15ms/step
Model accuracy: 25.0%

Accuracy for model_1: 0.25
Epoch 1/2
1/1 [==============================] - 1s 727ms/step - loss: 1.3711 - acc: 0.3125
Epoch 2/2
1/1 [==============================] - 0s 221ms/step - loss: 2.5417 - acc: 0.4062


INFO:tensorflow:Assets written to: model_2\assets


...

Saved model
Loaded model from disk
1/1 [==============================] - 0s 15ms/step
Model accuracy: 37.5%

Accuracy for model_9: 0.375
Epoch 1/10
3/5 [=================>............] - ETA: 0s - loss: 1.7647 - acc: 0.2159WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.


5/5 [==============================] - 1s 99ms/step - loss: 1.7647 - acc: 0.2159


INFO:tensorflow:Assets written to: model_10\assets


INFO:tensorflow:Assets written to: model_10\assets


Saved model
Loaded model from disk
1/1 [==============================] - 0s 15ms/step
Model accuracy: 75.0%

Accuracy for model_10: 0.75
Accuracies for all models: [0.25, 0.375, 0.375, 0.75, 0.625, 0.875, 0.75, 0.75, 0.375, 0.75]


In [26]:
pd.DataFrame(param_grid).assign(accuracy=accuracies)

,steps_per_epoch,epochs,model_name,accuracy
0,1,1,model_1,0.250
1,1,2,model_2,0.375
2,1,3,model_3,0.375
3,2,4,model_4,0.750
4,2,5,model_5,0.625
5,2,6,model_6,0.875
6,3,7,model_7,0.750
7,3,8,model_8,0.750
8,5,9,model_9,0.375
9,5,10,model_10,0.750


## Concepts

**1 - Discuss the effect of the following on accuracy and loss (train & test):** 
* **Increasing the steps_per_epoch**: this refers to the number of times the model updates weights within one epic. Increasing this value results in more weight updates (more opportunities to learn from the data). Increasing this value will generally improve the training accuracy and reduce the training loss. However, it might lead to overfitting if the model is learning too much from the training data without being able to generalize well to unseen data. The impact on test accuracy and loss depends on whether the model generalizes well to unseen data. If the model is overfitting due to a large number of steps_per_epoch, the test accuracy might not improve or could even decrease, while the test loss might increase. On the other hand, if the model is still able to generalize well, then increasing the steps_per_epoch could lead to better test accuracy and lower test loss.


* **Increasing the number of epochs**: this refers to the number of times the model iterates over the entire dataset. Increasing the value means the model will have more opportunities to learn from the whole dataset. As the number of epochs increases, the training accuracy will generally improve, and the training loss will decrease. This happens because the model is learning from the data over more iterations. However, there is a risk of overfitting if the number of epochs is too high, as the model may learn too much from the training data and fail to generalize to unseen data. The impact of increasing the number of epochs on test accuracy and loss depends on the model's ability to generalize to unseen data. If the model overfits due to too many epochs, the test accuracy might not improve or could even decrease, and the test loss might increase. However, if the model can generalize well, then increasing the number of epochs could lead to better test accuracy and lower test loss, up to a certain point. Beyond that point, the improvements will likely plateau, and further increasing the number of epochs might not have a significant impact.

**2 - Name two uses of zero padding in CNN**

* **Preserving spatial dimensions:** In a CNN, as the input passes through convolutional and pooling layers, the spatial dimensions (width and height) of the input tend to shrink. This reduction can sometimes result in very small feature maps, especially in deeper layers, which can limit the network's ability to learn complex and hierarchical features. By applying zero padding before convolution, the spatial dimensions are preserved, allowing the network to retain more information from the input and potentially learn more complex features. This is particularly useful when designing deep CNN architectures.


* **Controlling the field of view:** Zero padding can be used to control the receptive field or the field of view of the convolutional filters. With zero padding, the filters can slide over the edges and corners of the input, allowing the network to learn features from these regions. Without padding, the filters would only focus on the central parts of the input, potentially missing important information present at the edges and corners. By adjusting the amount of zero padding, the designer can control the degree to which the filters cover the input and thus influence the network's ability to learn features from different regions of the input.

**3 - What is the use of a 1 x 1 kernel in CNN**

* **Dimensionality reduction:** Reduces the number of channels while preserving spatial dimensions.
* **Increasing non-linearity:** Introduces additional non-linearity when followed by an activation function.
* **Feature map fusion:** Combines or fuses information from different channels.
* **Parameter efficiency:** Requires fewer parameters compared to larger kernel sizes, making it more computationally efficient.

**4 - What are the advantages of a CNN over a fully connected DNN for this image classification problem?**

* **Local feature extraction:** CNNs capture local patterns and hierarchically learn complex features.
* **Parameter efficiency:** CNNs have fewer trainable parameters due to shared weights in convolutional layers.
* **Translation invariance:** CNNs can recognize features regardless of their location in the image.
* **Scale and rotation invariance:** CNNs, combined with pooling layers, can learn features at different scales and rotations.
* **Exploiting spatial structure:** CNNs preserve and exploit spatial information, while fully connected DNNs treat input as flattened vectors.